In [ ]:
import os
from pathlib import Path
from fastai.imports import *  # imports the usual suspects

## Lesson 7 Collaborative Filtering

The video title is instead entitled "What's inside a neural net", which makes sense because it begins by continuing to go through "Road to the Top" [part 3](https://www.kaggle.com/code/jhoward/scaling-up-road-to-the-top-part-3) and part [part 4](https://www.kaggle.com/code/jhoward/multi-target-road-to-the-top-part-4).


### Gradient accumulation

* For large models, you will need a smaller batch size to fit in memory. However this will increase the variance of the gradient if done in the direct way. Instead you can accumulate the gradients over multiple batches before updating the weights. This is called gradient accumulation.

* In pytorch this can be done by simply calling  `loss.backward()` for each 'sub-batch' and then calling `optimizer.step()` after a number of sub-batches have been processed. For example, if you want a batch size of 64 but could not fit it in memmory, you could set the sub-batch size to 16 and accumulate gradients over 4 sub-batches. This works because the gradients are simply added together until you call `optimizer.zero_grad()`. (or otherwise zero the grads.)

* The results will be the same for most architectures, except for things that depend on the batch itself like batch normalization.  

* Fastai supports this directly with [Gradient Accumulation](https://docs.fast.ai/callback.training.html#gradientaccumulation) callback.

* Someone asked how you pick a batch size, and he says something like "people pcik as large a batch size as will fit in the GPU".  Which begs the question why use gradient accumulation at all?   Why not just decrease learning rate ?   Not clear, something something variance...  

* When experimenting with memory usage, and perhaps in general, it is a good idea to clear out memory between runs. You can do this by restarting the kernel, or by using `gc.collect()` and `torch.cuda.empty_cache()`.

* Notebook (part 3) also illustrates how he got ot the top by ensembling several different large models trained on different training sets (using train val split)

* Video at 38:00

### Multi-target training.


This section uses [part 4](https://www.kaggle.com/code/jhoward/multi-target-road-to-the-top-part-4) of the notebook.
He uses fast.ai's `DataBlock` to encapsulate the multiple target dataset.... the targets are the disease *and* the type of rice.

Since there are 10 diffrent diseases and 10 different varieties, he simply has the 'learner' output 20 different values. The first 10 will be used to predict the disease, and the second 10 will be used to predict the variety.   The loss function will add the loss for the two targets together. The loss uses F.cross_entropy seperatly for each target, which works because the ouptut of the model is a single vector of 20 logits (10 for each target).

The video also covers cross entropy loss and softmax, since he has not brought this up before. Why? Because `vision_learner` in simple case can figure it out automatically and has been using cross entropy under the hood.  `F.cross_entropy` in pytorch does the softmax to compute probabilities and then comptues the cross entropy loss: $\sum_{i} -y_i log(p_i)$ 

*sometimes* Multi-target training can help even if you really on care about one of the targets.  This is because the model can learn to use the other target as a kind of regularizer /  transfer learning.



### Collaborative Filtering

Video at 1:00:00

This section uses the [notebook](https://www.kaggle.com/code/jhoward/collaborative-filtering-deep-dive/notebook) which is also chapter 8 of the book.  He says he is presenting the chapter largely unaltered.


See the notebook for more, i will only put in highlights here.

* Data is tabular with user, movie, rating and timestamp. This is common type of data! 

* Idea is to predict the rating for a user and movie based on the ratings of other users and movies. 

* one way to do this is to create a user preference vector and movie vectors and score them by taking the dot product.  (If we were doing this manually, perhaps these could be characteristics of the movies and how strongly a user feels about these characterists.) This is called a dot product model. 

* The user vector and movie vector are learned by the model as latent factors / embeddings.   I.e. the users and movies are categorical, and we will learn embeddings for these categories.

* Jeremy uses the root mean square error between movie review predictiond actual to train the model.  First he illustrates this with excel model using the solver on a subset.

* Mathematically this is also called "matrix completion" since we have a matrix of movies vs users and we are trying to fill in the missing values. 

* note that looking up in an array is the same as a matrix multiply by a one-hot encoded vector.  We can think of the lookup as an optimizaiton

The model for this is super simple its just:

```

class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)
        
    def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        return sigmoid_range((users * movies).sum(dim=1), *self.y_range)
        
```

The sigmoid_range just forces the range to be 0 to 5. This is defined in fastai layers.py.
```
def sigmoid_range(x, low, high):
    "Sigmoid function with range `(low, high)`"
    return torch.sigmoid(x) * (high - low) + low
```


`Embedding` is just pytorch version of the embedding. 

#### Improvements

* add a bias term to the model.  This is just a single number for each user and movie.  This is added to the dot product.  This helps reflect that some users are more positive in gerneral or more negative , and that some movies are better or worse in general.


```python
class DotProductBias(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.user_bias = Embedding(n_users, 1)
        self.movie_factors = Embedding(n_movies, n_factors)
        self.movie_bias = Embedding(n_movies, 1)
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        res = (users * movies).sum(dim=1, keepdim=True)
        res += self.user_bias(x[:,0]) + self.movie_bias(x[:,1])
        return sigmoid_range(res, *self.y_range)
```

#### Regularization

* Weight decay or L2 regularization:  This is just adding the sum of the squares of the weights to the loss.  Why is it called weight decay?  Because when you take the gradient it looks like a friction term that pushes the weights towards zero. 

* The amount of regularization needs to be tuned.. if it is too small the model will overfit, if it is too large then it reduces the capacity of the model to learn. It is a hyperparameter called `wd` in fastai.  


#### More on embeddings

* The video ends here, but the chapter continues on, first implementing `Embedding` from scratch, which is just a table of parameters which can be indexed in the forward method.  Note that it is important to wrap the tensor in `nn.Parameter` so that it is included in the list of parameters that the optimizer updates.

* Then it goes on to look at the embeddings  and biases that the model learned.

* THe movies with the lowest bias are generally 'low rated' movies, and same goes for 'high bias' movies.  However it is more then just the average rating. A movie with a low bias is one that is rated low even though it is the type of movie you would generally like!  And high biased movies are movies you might like even if it is not the type of movie you generally like.  In the data the top movies where Titanic, Shawshank Redemption, Schinder's list, Silence of the Lambs and LA Confidential. 

* PCA can be used to try and understand the embeddings, or at least plot movie embeddings in 2 D. This is done in the notebook. The clustering is interesting.
In the book they use a fastai method to compute the pca. (fastai.torch_core)


#### Other notes

* Fastai does have a collab_learner that can do all of this in a few lines of code. 

* Movie embeddings can also be used to find movies that are "close to" other movies by using the L2 distance or cosine similarity.


* With new products or new users you have the 'bootstrap' problem. Some ideas:
    * Pick some typical element and use that embedding. 
    * Train a (tabular) model to predict embeddings based on other characteristics of the user or movie. This could based on questions you ask new users, for example. 
    * Representation bias is another issue: enthusiastic users are more likely to rate movies, and so the model will be biased towards them.  


### Deep Learning for Collaborative Filtering.

Last part of the notebook / chapter covered using deep learning for collaborative filtering. Basically instead of the dot product, we concatenate the user and movie embeddings and pass them through a neural network.  For example:

```python
class CollabNN(Module):
    def __init__(self, user_sz, item_sz, y_range=(0,5.5), n_act=100):
        self.user_factors = Embedding(*user_sz)
        self.item_factors = Embedding(*item_sz)
        self.layers = nn.Sequential(
            nn.Linear(user_sz[1]+item_sz[1], n_act),
            nn.ReLU(),
            nn.Linear(n_act, 1))
        self.y_range = y_range
        
    def forward(self, x):
        embs = self.user_factors(x[:,0]),self.item_factors(x[:,1])
        x = self.layers(torch.cat(embs, dim=1))
        return sigmoid_range(x, *self.y_range)
```



Collab_learner can also produce this kind of model if you pass in `use_nn=True`.   It basically uses `TabularModel` under the hood, since this is just what it is!

Which gives us a good use case for this: When we need to (or get to!) combine user reviews with other data about the user or movie.  